# Routing outputs to a specific cell


## About this notebook

This notebook belongs to a series of small projects which aim is to evaluate the [Jupyter](http://jupyter.org/) ecosystem for science experiments control. The main idea is use the _Juypter notebook_ as a convergence platform in order to offer a fully featured environment to scientists. 

## Topic of the day

When working in the notebook, we might have some asynchronous activity running in several cells. We could, for instance, have a couple of live plots updated regularly in their associated cell. In such a case, we certainly want any output generated by a given asynchronous process to be routed to the cell to which it's attached. However, so far, the notebook routes the outputs to the _current cell_ - i.e. to the one in which we are currently working. It means that any output produce from the python backend - e.g. messages or even some widgets - will drop into the _current cell_. That could be quite annoying. 

## The problem we want to address
The problem is simple: we want to be able to select the cell in which our outputs are diplayed/rendered/printed/...

By default, any 'new' output becomes part of the 'current' cell. Here is an example:

In [ ]:
print("this our 'target-cell'...")
print("... and this first message is an 'output' of 'target-cell'")

In [ ]:
print("we are now in another cell...")
print("... but we would like this message to also be a 'target-cell' output - how could we do that?")

Here is a solution based on [this expert trick](https://nbviewer.jupyter.org/gist/minrk/049545c1edcf20415bb3d68f16047628). A **big thanks** to [@minrk](https://github.com/minrk) for sharing his expertise. 

## CellContext class
A simple cell context holder (i.e. holds info related to the 'target-cell').

In [ ]:
import sys
from contextlib import wraps, contextmanager
from IPython import get_ipython

class CellContext(object):
    
    def __init__(self):
        k = get_ipython().kernel
        self._ident = k._parent_ident
        self._header = k._parent_header
        self._save_publish_status = k._publish_status
        k._publish_status = self.__disable_once
       
    def __disable_once(self, *args, **kw):
        get_ipython().kernel._publish_status = self._save_publish_status
        
    def __call__(self):
        return (self._ident, self._header)

## cell_context [python contextmanager]
A python context manager allowing to execute some user code in the context of a specific (or target) cell.

This `contextmanager` saves the current context, switches to the specified one, call `yield` to execute the user's then restore the initial context. It also flushes the standard streams in order to properly route `print` outputs. 

In [ ]:
@contextmanager
def cell_context(context):
    try:
        assert(isinstance(context, CellContext))
        kernel = get_ipython().kernel
        save_context = (kernel._parent_ident, kernel._parent_header)
        sys.stdout.flush()
        sys.stderr.flush()
        kernel.set_parent(*context())
    except Exception as e:
        print(e)
        return
    try:
        yield
    except:
        raise
    finally:
        sys.stdout.flush()
        sys.stderr.flush()
        kernel.set_parent(*save_context)

## NotebookCellContent class
This is supposed to become the super class of any object which outputs must be routed to a particular cell.
It notably provides cell specific API for `IPython.display` and `IPython.clear_output`. It also exposes a subset of  `logging.Logger` interface.

In [ ]:
import logging
from IPython.display import display, clear_output
    
class NotebookCellContent(object):

    def __init__(self):
        self._context = CellContext()
        self._logger = logging.getLogger('jupyter.for.controls')
        try:
            # no default handler workaround 
            h = self._logger.handlers[0]
        except IndexError:
            logging.basicConfig(format="[%(asctime)-15s] %(name)s: %(message)s", level=logging.ERROR)

    @property
    def context(self):
        return self._cell_context

    def display(self, widgets_layout):
        with cell_context(self._context):
            display(widgets_layout)
       
    def clear_output(self):
        with cell_context(self._context):
            clear_output()

    @property
    def logger(self):
        return self._logger

    def set_logging_level(self, level):
        self._logger.setLevel(level)

    def print(self, *args):
        with cell_context(self._context):
            print(*args)
            
    def debug(self, msg, *args, **kwargs):
        with cell_context(self._context):
            self._logger.debug(msg, *args, **kwargs)

    def info(self, msg, *args, **kwargs):
        with cell_context(self._context):
            self._logger.info(msg, *args, **kwargs)

    def warning(self, msg, *args, **kwargs):
        with cell_context(self._context):
            self._logger.warning(msg, *args, **kwargs)

    def error(self, msg, *args, **kwargs):
        with cell_context(self._context):
            self._logger.error(msg, *args, **kwargs)

    def critical(self, msg, *args, **kwargs):
        with cell_context(self._context):
            self._logger.critical(msg, *args, **kwargs)

    def exception(self, msg, *args, **kwargs):
        with cell_context(self._context):
            self._logger.exception(msg, *args, **kwargs)

## Let's play with a NotebookCellContent instance

In [ ]:
ncc = NotebookCellContent()
ncc.set_logging_level(logging.DEBUG)

In [ ]:
ncc.print("this simple print goes to 'ncc' cell")
ncc.debug("this debug message goes to 'ncc' cell")
print('this text goes to current cell')

##  An ipywigets example

In [ ]:
ncc = NotebookCellContent()

In [ ]:
import ipywidgets as ipw

def say_hello_to_the_world(attr):
    ncc.print('bonjour le monde!, hola el mundo!')

b = ipw.Button(description='Hello World!')
b.on_click(say_hello_to_the_world)

display(b)

In [ ]:
# we now want 'b' to also be displayed in 'ncc' cell
ncc.display(b)

## Conclusion
It seems we now know how to route any output to a specific cell. Hope this help...

Thanks again to @minrk.